<a href="https://colab.research.google.com/github/ARYANSRIVATSAVA/NLP/blob/main/Text_Similarity_using_siamese_networks_and_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
# Import Trax

!pip install -q -U trax
import trax

In [ ]:
import os
import nltk
from trax import layers as tl
from trax.supervised import training
from trax.fastmath import numpy as fastnp
import numpy as np
import pandas as pd
import random as rnd

# set random seeds
#trax.supervised.trainer_lib.init_random_number_generators(34)
rnd.seed(34)

In [ ]:
#from google.colab import files
#uploaded = files.upload()

In [ ]:
data = pd.read_csv("/content/train.csv")
N=len(data)
print('Number of question pairs: ', N)
data.head(10)

Number of question pairs:  404290


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [ ]:
N_train = 300000
N_test  = 10*1024
data_train = data[:N_train]
data_test  = data[N_train:N_train+N_test]
print("Train set:", len(data_train), "Test set:", len(data_test))
del(data) # remove to free memory

Train set: 300000 Test set: 10240


In [ ]:
td_index = (data_train['is_duplicate'] == 1).to_numpy()
td_index = [i for i, x in enumerate(td_index) if x]
print('number of duplicate questions: ', len(td_index))
print('indexes of first ten duplicate questions:', td_index[:10])

number of duplicate questions:  111473
indexes of first ten duplicate questions: [5, 7, 11, 12, 13, 15, 16, 18, 20, 29]


In [ ]:
print(data_train['question1'][5])
print(data_train['question2'][5])
print('is_duplicate: ', data_train['is_duplicate'][5])

Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?
I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?
is_duplicate:  1


In [ ]:
Q1_train_words = np.array(data_train['question1'][td_index])
Q2_train_words = np.array(data_train['question2'][td_index])

Q1_test_words = np.array(data_test['question1'])
Q2_test_words = np.array(data_test['question2'])
y_test  = np.array(data_test['is_duplicate'])

In [ ]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> punkt
Command 'punkt' unrecognized

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt
      Unzipping tokenizers/punkt.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [ ]:
#create arrays
Q1_train = np.empty_like(Q1_train_words)
Q2_train = np.empty_like(Q2_train_words)

Q1_test = np.empty_like(Q1_test_words)
Q2_test = np.empty_like(Q2_test_words)

In [ ]:




# Building the vocabulary with the train set         (this might take a minute)
from collections import defaultdict
#punkt module has to be downloaded using nltk.download()
vocab = defaultdict(lambda: 0)
vocab['<PAD>'] = 1

for idx in range(len(Q1_train_words)):
    Q1_train[idx] = nltk.word_tokenize(Q1_train_words[idx])
    Q2_train[idx] = nltk.word_tokenize(Q2_train_words[idx])
    q = Q1_train[idx] + Q2_train[idx]
    for word in q:
        if word not in vocab:
            vocab[word] = len(vocab) + 1
print('The length of the vocabulary is: ', len(vocab))

The length of the vocabulary is:  36352


In [ ]:
print(Q1_train[5])
print(Q2_train[5])


['What', 'would', 'a', 'Trump', 'presidency', 'mean', 'for', 'current', 'international', 'master', '’', 's', 'students', 'on', 'an', 'F1', 'visa', '?']
['How', 'will', 'a', 'Trump', 'presidency', 'affect', 'the', 'students', 'presently', 'in', 'US', 'or', 'planning', 'to', 'study', 'in', 'US', '?']


In [ ]:
print(vocab['<PAD>'])
print(vocab['Astrology'])
print(vocab['Astronomy'])

1
2
0


In [ ]:
for idx in range(len(Q1_test_words)):
    Q1_test[idx] = nltk.word_tokenize(Q1_test_words[idx])
    Q2_test[idx] = nltk.word_tokenize(Q2_test_words[idx])

In [ ]:
print('Train set has reduced to: ', len(Q1_train) )
print('Test set length: ', len(Q1_test) )

Train set has reduced to:  111473
Test set length:  10240


In [ ]:
# Converting questions to array of integers
for i in range(len(Q1_train)):
    Q1_train[i] = [vocab[word] for word in Q1_train[i]]
    Q2_train[i] = [vocab[word] for word in Q2_train[i]]


for i in range(len(Q1_test)):
    Q1_test[i] = [vocab[word] for word in Q1_test[i]]
    Q2_test[i] = [vocab[word] for word in Q2_test[i]]

In [ ]:
print('first question in the train set:\n')
print(Q1_train_words[0], '\n')
print('encoded version:')
print(Q1_train[0],'\n')

print('first question in the test set:\n')
print(Q1_test_words[0], '\n')
print('encoded version:')
print(Q1_test[0])

first question in the train set:

Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me? 

encoded version:
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21] 

first question in the test set:

What were some of the troubles you have faced during and after your 9 months period of pregnancy? 

encoded version:
[30, 271, 116, 131, 78, 28882, 53, 218, 6595, 124, 11, 267, 56, 1636, 606, 2091, 131, 4329, 21]


In [ ]:
cut_off = int(len(Q1_train)*.8)
train_Q1, train_Q2 = Q1_train[:cut_off], Q2_train[:cut_off]
val_Q1, val_Q2 = Q1_train[cut_off: ], Q2_train[cut_off:]
print('Number of duplicate questions: ', len(Q1_train))
print("The length of the training set is:  ", len(train_Q1))
print("The length of the validation set is: ", len(val_Q1))

Number of duplicate questions:  111473
The length of the training set is:   89178
The length of the validation set is:  22295


In [ ]:
def data_generator(Q1, Q2, batch_size, pad=0, shuffle=True):
    """Generator function that yields batches of data

    Args:
        Q1 (list): List of transformed (to tensor) questions.
        Q2 (list): List of transformed (to tensor) questions.
        batch_size (int): Number of elements per batch.
        pad (int, optional): Pad character from the vocab. Defaults to 1.
        shuffle (bool, optional): If the batches should be randomnized or not. Defaults to True.
    Yields:
        tuple: Of the form (input1, input2) with types (numpy.ndarray, numpy.ndarray)
        NOTE: input1: inputs to your model [q1a, q2a, q3a, ...] i.e. (q1a,q1b) are duplicates
              input2: targets to your model [q1b, q2b,q3b, ...] i.e. (q1a,q2i) i!=a are not duplicates
    """

    input1 = []
    input2 = []
    idx = 0
    len_q = len(Q1)
    question_indexes = [*range(len_q)]

    if shuffle:
        rnd.shuffle(question_indexes)

    #Assuming true condition
    while True:
        if idx >= len_q:
            # if idx is greater than or equal to len_q, set idx accordingly
            # (Hint: look at the instructions above)
            idx = 0
            # shuffle to get random batches if shuffle is set to True
            if shuffle:
                rnd.shuffle(question_indexes)

        # get questions at the `question_indexes[idx]` position in Q1 and Q2
        q1 = Q1[question_indexes[idx]]
        q2 = Q2[question_indexes[idx]]

        # increment idx by 1
        idx += 1
        # append q1
        input1.append(q1)
        # append q2
        input2.append(q2)
        if len(input1) == batch_size:

            max_len = max(max([len(q) for q in input1]),
                          max([len(q) for q in input2]))
            # pad to power-of-2 (Hint: look at the instructions above)
            max_len = 2**int(np.ceil(np.log2(max_len)))
            b1 = []
            b2 = []
            for q1, q2 in zip(input1, input2):
                # add [pad] to q1 until it reaches max_len
                q1 = q1 + [pad] * (max_len - len(q1))
                q2 = q2 + [pad] * (max_len - len(q2))
                # append q1
                b1.append(q1)
                b2.append(q2)
            # use b1 and b2
            yield np.array(b1), np.array(b2)

            # reset the batches
            input1, input2 = [], []

In [ ]:
batch_size = 2
res1, res2 = next(data_generator(train_Q1, train_Q2, batch_size))
print("First questions  : ",'\n', res1, '\n')
print("Second questions : ",'\n', res2)

First questions  :  
 [[   30    87    78  1016    39  1608  1609    21     0     0     0     0
      0     0     0     0]
 [   32   545    73   230 10094   229   231    21     0     0     0     0
      0     0     0     0]] 

Second questions :  
 [[  30  156   78 3940 1918   39 1608 1609   21    0    0    0    0    0
     0    0]
 [  32   38    4  229  230  231   28 2582    6  816   21    0    0    0
     0    0]]


In [ ]:
def Siamese(vocab_size=len(vocab), d_model=128, mode='train'):

    def normalize(x):  # normalizes the vectors to have L2 norm 1
        return x / fastnp.sqrt(fastnp.sum(x * x, axis=-1, keepdims=True))

   q_processor = tl.Serial(  # Processor will run on Q1 and Q2.
        tl.Embedding(vocab_size, d_model),
        # Run LSTM. If this is not dim d_model it raises an error
        tl.LSTM(d_model),
        # Average vectors on the length axis.
        tl.Mean(axis=1),
        tl.Fn('Normalize', lambda x: normalize(x))  # Apply normalize function
    )

    # Run on Q1 and Q2 in parallel.
    model = tl.Parallel(q_processor, q_processor)
    return model

In [ ]:
model = Siamese()
print(model)

Parallel_in2_out2[
  Serial[
    Embedding_41794_128
    LSTM_128
    Mean
    Normalize
  ]
  Serial[
    Embedding_41794_128
    LSTM_128
    Mean
    Normalize
  ]
]


In [ ]:
def TripletLossFn(v1, v2, margin=0.25):


    # calculate new batch size
    batch_size = len(scores)
    # use fastnp to grab all postive `diagonal` entries in `scores`
    positive = fastnp.diagonal(scores)  # the positive ones (duplicates)
    # multiply `fastnp.eye(batch_size)` with 2.0 and subtract it out of `scores`
    negative_without_positive = scores - 2.0 * fastnp.eye(batch_size)
    # take the row by row `max` of `negative_without_positive`.
    closest_negative = negative_without_positive.max(axis=1) # [batch]
    # subtract `fastnp.eye(batch_size)` out of 1.0 and do element-wise multiplication with `scores`
    negative_zero_on_duplicate = scores * (1.0 - fastnp.eye(batch_size))
    # use `fastnp.sum` on `negative_zero_on_duplicate` for `axis=1` and divide it by `(batch_size - 1)`
    mean_negative = np.sum(negative_zero_on_duplicate, axis=1) / (batch_size-1)
    # compute `fastnp.maximum` among 0.0 and `A`
    # A = subtract `positive` from `margin` and add `closest_negative`
    triplet_loss1 = fastnp.maximum(0.0, margin - positive + closest_negative)
    # compute `fastnp.maximum` among 0.0 and `B`
    # B = subtract `positive` from `margin` and add `mean_negative`
    triplet_loss2 = fastnp.maximum(0.0, margin - positive + mean_negative)
    # add the two losses together and take the `fastnp.mean` of it
    triplet_loss = fastnp.mean(triplet_loss1 + triplet_loss2)

    return triplet_loss

In [ ]:
v1 = np.array([[0.26726124, 0.53452248, 0.80178373],[0.5178918 , 0.57543534, 0.63297887]])
v2 = np.array([[ 0.26726124,  0.53452248,  0.80178373],[-0.5178918 , -0.57543534, -0.63297887]])
TripletLossFn(v2,v1)
print("Triplet Loss:", TripletLossFn(v2,v1))

Triplet Loss: 0.5


In [ ]:
from functools import partial
#Partial function gives default values to parameters to check if the function is working correctly
def TripletLoss(margin=0.25):
    triplet_loss_fn = partial(TripletLossFn, margin=margin)
    return tl.Fn('TripletLoss', triplet_loss_fn)

In [ ]:
batch_size = 256
train_generator = data_generator(train_Q1, train_Q2, batch_size, vocab['<PAD>'])
val_generator = data_generator(val_Q1, val_Q2, batch_size, vocab['<PAD>'])
print('train_Q1.shape ', train_Q1.shape)
print('val_Q1.shape   ', val_Q1.shape)

train_Q1.shape  (89178,)
val_Q1.shape    (22295,)


In [ ]:
lr_schedule = trax.lr.warmup_and_rsqrt_decay(400, 0.01)

def train_model(Siamese, TripletLoss, lr_schedule, train_generator=train_generator, val_generator=val_generator, output_dir='model/'):

    output_dir = os.path.expanduser(output_dir)

    train_task = training.TrainTask(
        labeled_data=train_generator,
        loss_layer=TripletLoss(),
        optimizer=trax.optimizers.Adam(0.01),
        lr_schedule=lr_schedule,
    )

    eval_task = training.EvalTask(
        labeled_data=val_generator,
        metrics=[TripletLoss()],
    )

    training_loop = training.Loop(Siamese(),
                                  train_task,
                                  eval_tasks=eval_task,
                                  output_dir=output_dir)

    return training_loop

In [ ]:
train_steps = 1000
training_loop = train_model(Siamese, TripletLoss, lr_schedule)
training_loop.run(train_steps)


/usr/local/lib/python3.7/dist-packages/jax/_src/lib/xla_bridge.py:435: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  "jax.host_count has been renamed to jax.process_count. This alias "



Step    400: Ran 100 train steps in 87.94 secs
Step    400: train TripletLoss |  0.49996403
Step    400: eval  TripletLoss |  0.49993986

Step    500: Ran 100 train steps in 84.21 secs
Step    500: train TripletLoss |  0.43893430
Step    500: eval  TripletLoss |  0.38584650

Step    600: Ran 100 train steps in 82.73 secs
Step    600: train TripletLoss |  0.33849263
Step    600: eval  TripletLoss |  0.32622865

Step    700: Ran 100 train steps in 79.96 secs
Step    700: train TripletLoss |  0.30007324
Step    700: eval  TripletLoss |  0.27818257

Step    800: Ran 100 train steps in 81.20 secs
Step    800: train TripletLoss |  0.27914274
Step    800: eval  TripletLoss |  0.27767241

Step    900: Ran 100 train steps in 78.76 secs
Step    900: train TripletLoss |  0.26637343
Step    900: eval  TripletLoss |  0.25478059

Step   1000: Ran 100 train steps in 83.86 secs
Step   1000: train TripletLoss |  0.22693928
Step   1000: eval  TripletLoss |  0.19278604

Step   1100: Ran 100 train steps 

In [ ]:
model1 = Siamese()
model1.init_from_file('/content/model.pkl.gz')

(((array([[-0.7434783 , -0.5323    ,  0.26556844, ...,  0.24734499,
            0.971258  , -0.3176002 ],
          [-1.9103315 , -1.2298064 ,  0.7929189 , ..., -1.3576206 ,
           -0.9268899 ,  0.11710498],
          [ 1.1356051 ,  1.2533569 ,  1.4670613 , ...,  1.2557949 ,
            1.1703947 ,  1.7554839 ],
          ...,
          [-0.49271938,  0.06522572, -0.74080336, ...,  1.4723355 ,
            1.1603701 ,  0.51038134],
          [ 1.4607905 , -0.15703319,  0.5001072 , ...,  0.18419997,
           -0.5392223 , -0.4307455 ],
          [-1.4316021 , -1.2368174 ,  0.0611912 , ..., -0.24021208,
            0.34730613, -0.07061554]], dtype=float32),
   (((), ((), ())),
    ((array([[-0.02550941, -0.06643244, -0.03194237, ..., -0.01651929,
              -0.0235158 , -0.02485679],
             [-0.0683428 , -0.06671927,  0.0349182 , ...,  0.01044205,
               0.02273431,  0.0717328 ],
             [-0.03171944,  0.01028203,  0.05781721, ..., -0.04236581,
              -0.

In [ ]:
def predict(question1, question2, threshold, model, vocab, data_generator=data_generator, verbose=False):

    q1 = nltk.word_tokenize(question1)  # tokenize
    q2 = nltk.word_tokenize(question2)  # tokenize
    Q1, Q2 = [], []
    for word in q1:  # encode q1
        Q1 += [vocab[word]]
    for word in q2:  # encode q2
        Q2 += [vocab[word]]

    Q1, Q2 = next(data_generator([Q1], [Q2], 1, vocab['<PAD>']))
    # Call the model
    v1, v2 = model1((Q1, Q2))
    d = np.dot(v1[0], v2[0].T)
        res = d > threshold

    if(verbose):
        print("Q1  = ", Q1, "\nQ2  = ", Q2)
        print("d   = ", d)
        print("res = ", res)

    return res

In [ ]:
! pip install transformers

     |████████████████████████████████| 4.0 MB 4.5 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████████████████████| 6.6 MB 37.1 MB/s 
     |████████████████████████████████| 895 kB 48.9 MB/s 
     |████████████████████████████████| 596 kB 48.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
% pip install sentencepiece

     |████████████████████████████████| 1.2 MB 3.9 MB/s 


In [ ]:
pip install transformers torch sentencepiece

In [ ]:
from transformers import pipeline

# using pipeline API for summarization task
summarization = pipeline("summarization") #pipeline is a model,summarization is a method in it
original_text = """
Paul Walker is hardly the first actor to die during a production.
But Walker's death in November 2013 at the age of 40 after a car crash was especially eerie given his rise to fame in the "Fast and Furious" film franchise.
The release of "Furious 7" on Friday offers the opportunity for fans to remember -- and possibly grieve again -- the man that so many have praised as one of the nicest guys in Hollywood.
"He was a person of humility, integrity, and compassion," military veteran Kyle Upham said in an email to CNN.
Walker secretly paid for the engagement ring Upham shopped for with his bride.
"We didn't know him personally but this was apparent in the short time we spent with him.
I know that we will never forget him and he will always be someone very special to us," said Upham.
The actor was on break from filming "Furious 7" at the time of the fiery accident, which also claimed the life of the car's driver, Roger Rodas.
Producers said early on that they would not kill off Walker's character, Brian O'Connor, a former cop turned road racer. Instead, the script was rewritten and special effects were used to finish scenes, with Walker's brothers, Cody and Caleb, serving as body doubles.
There are scenes that will resonate with the audience -- including the ending, in which the filmmakers figured out a touching way to pay tribute to Walker while "retiring" his character. At the premiere Wednesday night in Hollywood, Walker's co-star and close friend Vin Diesel gave a tearful speech before the screening, saying "This movie is more than a movie." "You'll feel it when you see it," Diesel said. "There's something emotional that happens to you, where you walk out of this movie and you appreciate everyone you love because you just never know when the last day is you're gonna see them." There have been multiple tributes to Walker leading up to the release. Diesel revealed in an interview with the "Today" show that he had named his newborn daughter after Walker.
Social media has also been paying homage to the late actor. A week after Walker's death, about 5,000 people attended an outdoor memorial to him in Los Angeles. Most had never met him. Marcus Coleman told CNN he spent almost $1,000 to truck in a banner from Bakersfield for people to sign at the memorial. "It's like losing a friend or a really close family member ... even though he is an actor and we never really met face to face," Coleman said. "Sitting there, bringing his movies into your house or watching on TV, it's like getting to know somebody. It really, really hurts." Walker's younger brother Cody told People magazine that he was initially nervous about how "Furious 7" would turn out, but he is happy with the film. "It's bittersweet, but I think Paul would be proud," he said. CNN's Paul Vercammen contributed to this report.
"""
summary_text1 = summarization(original_text)[0]['summary_text'] # giving input
print("Summary:", summary_text1)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Summary:  Paul Walker died in November 2013 after a car crash in Los Angeles . The late actor was one of the nicest guys in Hollywood . The release of "Furious 7" on Friday offers a chance to grieve again . There have been multiple tributes to Walker leading up to the film's release .


In [ ]:
summary_text1

' Paul Walker died in November 2013 after a car crash in Los Angeles . The late actor was one of the nicest guys in Hollywood . The release of "Furious 7" on Friday offers a chance to grieve again . There have been multiple tributes to Walker leading up to the film\'s release .'

In [ ]:
original_text = """Paul Walker met an untimely death.

Known for being a generous philanthropist, Walker had spent November 30, 2013, at a toy drive event for his disaster relief charity, Reach Out Worldwide, which was founded in the wake of the 2010 earthquake in Haiti. Walker left happily just before 3:30 p.m. — and he was never seen alive again.The pair left the event in a 2005 Porsche Carrera GT, with Rodas driving and Walker riding shotgun. The car was known for being hard to handle, and only a few hundred yards away from the shop, Rodas lost control of the vehicle. The Porsche was traveling at about 100 miles per hour before it hit a curb, a tree, a light post, and then another tree before bursting into flames."""

In [ ]:
summary_text2 = summarization(original_text)[0]['summary_text']
print("Summary:", summary_text2)

Summary:  Paul Walker died in November 2013 after a car crash in Los Angeles . The late actor was one of the nicest guys in Hollywood . The release of "Furious 7" on Friday offers a chance to grieve again . There have been multiple tributes to Walker leading up to the film's release .


In [ ]:

# Python3 code to demonstrate
# removal of bad_chars
# using replace()

# initializing bad_chars_list
bad_chars = [';', ':', '!','-','"',"*"]

# initializing test string
test_string1 = summary_text1
test_string2 = summary_text2
# printing original string
print ("Original String : " + test_string1)
print ("Original String : " + test_string2)
# using replace() to
# remove bad_chars
for i in bad_chars :
    test_string = test_string.replace(i, '')

# printing resultant string
print ("Resultant string1 is : " + test_string1)
print ("Resultant string2 is : " + test_string2 )

Original String :  Paul Walker died in November 2013 after a car crash in Los Angeles . The late actor was one of the nicest guys in Hollywood . The release of "Furious 7" on Friday offers a chance to grieve again . There have been multiple tributes to Walker leading up to the film's release .
Original String :  Paul Walker died in November 2013 after a car crash in Los Angeles . The late actor was one of the nicest guys in Hollywood . The release of "Furious 7" on Friday offers a chance to grieve again . There have been multiple tributes to Walker leading up to the film's release .
Resultant string1 is :  Paul Walker died in November 2013 after a car crash in Los Angeles . The late actor was one of the nicest guys in Hollywood . The release of "Furious 7" on Friday offers a chance to grieve again . There have been multiple tributes to Walker leading up to the film's release .
Resultant string2 is :  Paul Walker died in November 2013 after a car crash in Los Angeles . The late actor wa

In [ ]:
question1 = test_string1
question2 = test_string2

predict(question1 , question2, 0.7, model1, vocab, verbose = True)

Q1  =  [[ 9030 13394  1870    28  1872  5221   267     6  1708  6584    28  9521
   9522   148  2366   960  4068    55   111   131    78  1857  2059    28
    750   148  2366  3398   131   619     0  1300   622    72  4159  3622
      6  1019    39     0  7287   148  5799   218  1496  2816     0    39
  13394  6870   495    39    78  2061   254  3398   148     1     1     1
      1     1     1     1]] 
Q2  =  [[ 9030 13394  1870    28  1872  5221   267     6  1708  6584    28  9521
   9522   148  2366   960  4068    55   111   131    78  1857  2059    28
    750   148  2366  3398   131   619     0  1300   622    72  4159  3622
      6  1019    39     0  7287   148  5799   218  1496  2816     0    39
  13394  6870   495    39    78  2061   254  3398   148     1     1     1
      1     1     1     1]]
d   =  1.0
res =  True


True

In [ ]:
#AS BOTH THE PARAGRAPHS ARE OF SAME CONTEXT THE SIMILARITY IS RESULTED IN 1(MAX).